In [1]:
import datasets

In [2]:
from datasets import load_dataset
dataset = load_dataset('json', data_files="simplified-nq-train.jsonl")

In [4]:
sample = dataset['train'][1]

In [6]:
doc = sample['document_text']

In [7]:
sample['annotations'][0]['long_answer']

{'start_token': 212, 'candidate_index': 15, 'end_token': 310}

In [8]:
init = sample['annotations'][0]['long_answer']['start_token']
end = sample['annotations'][0]['long_answer']['end_token']

In [10]:
" ".join(doc.split(" ")[init:end])

"<P> Tracy McConnell , better known as `` The Mother '' , is the title character from the CBS television sitcom How I Met Your Mother . The show , narrated by Future Ted , tells the story of how Ted Mosby met The Mother . Tracy McConnell appears in 8 episodes from `` Lucky Penny '' to `` The Time Travelers '' as an unseen character ; she was first seen fully in `` Something New '' and was promoted to a main character in season 9 . The Mother is played by Cristin Milioti . </P>"

In [11]:
sample['question_text']

'how i.met your mother who is the mother'

In [12]:
from transformers import pipeline
pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-tc-big-en-pt")
print(pipe(">>por<< Tom tried to stab me."))

C:\Users\dudua\Desktop\Home\IC\MyCodes\.env\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\dudua\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


C:\Users\dudua\Desktop\Home\IC\MyCodes\.env\lib\site-packages\transformers\models\marian\tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


[{'translation_text': 'O Tom tentou esfaquear-me.'}]


In [32]:
small_set = dataset['train']

In [69]:
import pandas as pd

In [75]:
df = pd.DataFrame(columns = ['title', 'pergunta', 'answer'])

In [76]:
for i in range(10):
    doc = small_set[i]['document_text']
    init = small_set[i]['annotations'][0]['long_answer']['start_token']
    end = small_set[i]['annotations'][0]['long_answer']['end_token']
    question = small_set[i]['question_text']
    answer = " ".join(doc.split(" ")[init:end])
    answer= pipe('>>por<< ' + answer)
    new_row = {'title': [doc], 'pergunta': [question], 'answer': [answer]}
    df_aux = pd.DataFrame(new_row)
    df = pd.concat([df, df_aux])




In [77]:
df

,title,pergunta,answer
0,Email marketing - Wikipedia <H1> Email marketi...,which is the most common use of opt-in e-mail ...,[{'translation_text': 'P> Um exemplo comum de ...
0,The Mother ( How I Met Your Mother ) - wikiped...,how i.met your mother who is the mother,"[{'translation_text': 'P > Tracy McConnell , m..."
0,Human fertilization - wikipedia <H1> Human fer...,what type of fertilisation takes place in humans,[{'translation_text': 'P> O processo de fertil...
0,List of National Football League career quarte...,who had the most wins in the nfl,[{'translation_text': 'P> O quarterback ativo ...
0,Roanoke Colony - wikipedia <H1> Roanoke Colony...,what happened to the lost settlement of roanoke,[{'translation_text': 'E-mail'}]
0,List of regions of Africa - wikipedia <H1> Lis...,what are the different regions of africa and h...,[{'translation_text': 'E-mail'}]
0,Pom Klementieff - wikipedia <H1> Pom Klementie...,who played mantis guardians of the galaxy 2,[{'translation_text': 'P > Pom Klementieff (na...
0,Frosty the Snowman ( film ) - wikipedia <H1> F...,who did the voice of the magician in frosty th...,[{'translation_text': 'E-mail'}]
0,History of the Acadians - wikipedia <H1> Histo...,what indian tribe did the acadians form friend...,[{'translation_text': 'P> A sobrevivência dos ...
0,Outer Banks - wikipedia <H1> Outer Banks </H1>...,what is considered the outer banks in north ca...,[{'translation_text': 'E-mail'}]


In [48]:
small_set[i]['question_text']

'which is the most common use of opt-in e-mail marketing'

In [30]:
for i in dataset['train'][1:3]:
    print(i)

document_text
long_answer_candidates
question_text
annotations
document_url
example_id
